In [8]:
# REPARACIÓN DE DATOS
import pandas as pd
from datasets import Dataset

# Cargar Excel
df = pd.read_excel('1_dataset_chatbot.xlsx')

# Crear una columna con el formato que Llama-3 entiende (Alpaca/Instruct)

system_prompt = "Eres Cantor, un asistente poético y emocional. Responde inspirándote en esta canción."

def format_data(row):
    # Simulamos una interacción.
    # Entrada: El usuario pide consejo o habla de un tema.
    # Salida: El verso de la canción.

    # NOTA: Para un fine-tuning real de alta calidad, idealmente tendrías preguntas variadas.
    # Aquí usaremos un truco: asumimos que el usuario pregunta sobre el tema de la canción.

    instruction = f"Háblame con el sentimiento de la canción '{row['nombre_cancion']}'."
    output = str(row['estrofa']) # Aseguramos que sea string

    # Formato Alpaca (estándar para enseñar instrucciones)
    text = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{system_prompt} {instruction}

### Response:
{output}<|end_of_text|>"""

    return text

# Crear columna de texto formateado
df['text'] = df.apply(format_data, axis=1)

# Convertir a formato HuggingFace Dataset
dataset = Dataset.from_pandas(df[['text']])

# Ver un ejemplo de cómo quedó
print(dataset[0]['text'])

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Eres Cantor, un asistente poético y emocional. Responde inspirándote en esta canción. Háblame con el sentimiento de la canción '26 de mayo'.

### Response:
el veintiseis del mes de mayo nacio un niñito en el año cincuenta y siete y alla en la junta fue bautizado y hoy se conoce con el nombre de diomedes.<|end_of_text|>


In [1]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()

# Esta lógica instala la versión correcta de Unsloth dependiendo de tu GPU en Colab
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Para GPUs nuevas (A100, H100)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Para GPUs estándar de Colab (T4 - La que probablemente usas)
    !pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
from huggingface_hub import login
from google.colab import userdata

# Opción A: Si guardaste el token en los "Secretos" de Colab (la llavecita a la izquierda) con nombre 'HF_TOKEN'
try:
    login(token=userdata.get('HF_TOKEN'))
except:
    # Opción B: Login manual (te pedirá pegar el token en una cajita)
    login()

In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 2048
dtype = None
load_in_4bit = True

print("Cargando modelo base... (esto puede tardar unos minutos la primera vez)")

try:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "unsloth/llama-3-8b-bnb-4bit",
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    print("¡Modelo cargado exitosamente!")
except Exception as e:
    print(f"Error al cargar: {e}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: Could not find Config class in trl.trainer.dpo_trainer. Found: []
Unsloth: Could not find Config class in trl.trainer.iterative_sft_trainer. Found: []
Unsloth: Could not find Config class in trl.trainer.sft_trainer. Found: []
Cargando modelo base... (esto puede tardar unos minutos la primera vez)
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
¡Modelo cargado exitosamente!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

print("Adaptadores PEFT aplicados correctamente.")

Unsloth 2025.11.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Adaptadores PEFT aplicados correctamente.


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Configuración del entrenamiento
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Cambiado a False por seguridad (True a veces da problemas con datasets pequeños)
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Pon 60 para probar rápido, luego sube a 300-500
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # <--- ESTO SOLUCIONA EL AVISO DE WANDB
    ),
)

# Iniciar entrenamiento
print("Iniciando entrenamiento...")
trainer_stats = trainer.train()
print("¡Entrenamiento finalizado!")

Map (num_proc=2):   0%|          | 0/1816 [00:00<?, ? examples/s]

Iniciando entrenamiento...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,816 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.643000
2,3.598200
3,3.575600
4,3.655200
5,3.410400
6,3.248900
7,3.352100
8,3.304300
9,3.038300
10,3.027400


¡Entrenamiento finalizado!


In [19]:
import textwrap

def hablar_con_cantor(pregunta):
    FastLanguageModel.for_inference(model)

    # Cambiamos un poco el prompt para pedir explícitamente una reflexión
    prompt = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Eres Cantor. Responde a esta situación con una reflexión profunda y varios versos de tus canciones: {pregunta}

### Response:
"""

    inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens = 256,    # Damos espacio para respuestas largas
        min_new_tokens = 50,     # <--- OBLIGAMOS a que escriba al menos 30 "palabras/tokens"
        repetition_penalty = 1.3, # <--- IMPORTANTE: Evita el "me siento solo, me siento solo"
        temperature = 0.7,       # Un poco más de creatividad
        use_cache = True,
    )

    respuesta = tokenizer.batch_decode(outputs)[0]
    return respuesta.split("### Response:")[-1].replace("<|end_of_text|>", "").strip()

In [20]:
print(textwrap.fill(hablar_con_cantor("¿Qué puedo hacer para superar el desamor?"), width=80))

en la vida no hay que amargarse por algo porque lo bueno puede venir luego del
malo pero si es un amor perdido ya nunca mas se podra recuperarlo en su lugar yo
me voy despacio sin volverme atrás mi corazon fue herido como siempre cuando
ella estaba junto al otro


In [21]:
print(textwrap.fill(hablar_con_cantor("¿Qué se necesita para ser feliz"), width=80))

un amor verdadero sin mentiras ni engaños, el cielo es mi hogar pues tengo un
buen hermano que me inspirara en la vida y no olvidarlo nunca por eso quiero
dedicarme al cantante poeta porque este dia ya llegue


In [22]:
print(textwrap.fill(hablar_con_cantor("¿Qué hacer con la infidelidad?"), width=80))

Hoy he decidido que no voy a volver más, porque mi amor me ha traicionado como
un asesino. Ahora ya estoy desilusionada por esa mujer tan malvadezca que se
acostó en el lecho ajeno


In [23]:
print(textwrap.fill(hablar_con_cantor("¿Para qué son los amigos?"), width=80))

que sin saberlo el amigo me va ayudando, porque le hago ver la verdad que no hay
quien pueda hacerme mal ni desgraciarme. por eso digo yo mi amiga o si prefieres
tu hermanita te ayudo en todo lo que necesitas


In [24]:
print(textwrap.fill(hablar_con_cantor("¿Tomar cerveza da felicidad?"), width=80))

una vez tomé un trago para que me sentara el dolor pero no se apagó la luz del
amor todavía siento esa herida en mi corazón es como si ella nunca hubiera
estado aquí entonces toma otro tragito porque nos va a emocionarse por dentro
